# Agent Hypervisor — Interactive Exploration

This notebook walks through the core features of the **agent-hypervisor** runtime:

| Section | Feature |
|---------|--------|
| 1 | Setup & imports |
| 2 | Execution rings & privilege levels |
| 3 | Saga pattern with compensation |
| 4 | Kill switch activation |
| 5 | Resource / rate limiting |
| 6 | Hash-chained audit trail |
| 7 | Joint liability — vouching & slashing |
| 8 | Visualization |

> **Prerequisite:** install the package in editable mode:  
> ```bash
> pip install -e ".[dev]" plotly
> ```

---
## 1 — Setup

In [ ]:
import asyncio
import sys, pathlib, datetime as dt

# Ensure the package is importable when running from the notebooks/ folder
ROOT = pathlib.Path.cwd().parent
if str(ROOT / "src") not in sys.path:
    sys.path.insert(0, str(ROOT / "src"))

from hypervisor import (
    Hypervisor,
    SessionConfig,
    ConsistencyMode,
    ExecutionRing,
    ReversibilityLevel,
)
from hypervisor.models import ActionDescriptor, SessionParticipant
from hypervisor.saga import SagaOrchestrator, Saga, SagaState, StepState
from hypervisor.security.kill_switch import KillSwitch, KillReason
from hypervisor.liability.vouching import VouchingEngine
from hypervisor.liability.slashing import Slashing
from hypervisor.audit import DeltaEngine, SemanticDelta, VFSChange
from hypervisor.rings import RingEnforcer, ActionClassifier

print(f"Hypervisor package loaded from {ROOT / 'src'}")
print(f"Execution rings: {[r.name for r in ExecutionRing]}")

### Helper — run async code in notebook cells

In [ ]:
def run(coro):
    """Run an async coroutine in the notebook."""
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        loop = None
    if loop and loop.is_running():
        import nest_asyncio
        nest_asyncio.apply()
    return asyncio.get_event_loop().run_until_complete(coro)

### Create a Hypervisor and Session

In [ ]:
hv = Hypervisor()

config = SessionConfig(
    consistency_mode=ConsistencyMode.STRONG,
    max_participants=10,
    max_duration_seconds=3600,
    min_eff_score=0.3,
    enable_audit=True,
)

session = run(hv.create_session(config, creator_did="agent-root"))
SESSION_ID = session.session.session_id
print(f"Session created: {SESSION_ID}")
print(f"State: {session.session.state}")

---
## 2 — Execution Rings

Agents are assigned to **rings 0–3** based on their *effective trust score*:

| Ring | Level | EFF score requirement |
|------|-------|----------------------|
| 0 | Root | Creator only |
| 1 | Privileged | ≥ 0.70 + consensus |
| 2 | Standard | ≥ 0.40 |
| 3 | Sandbox | < 0.40 |

In [ ]:
# Define some sample actions with varying reversibility
read_action = ActionDescriptor(
    action_id="read-data",
    name="Read dataset",
    execute_api="/api/data/read",
    undo_api=None,
    reversibility=ReversibilityLevel.FULL,
    is_read_only=True,
)

write_action = ActionDescriptor(
    action_id="write-report",
    name="Write report",
    execute_api="/api/reports/write",
    undo_api="/api/reports/undo",
    reversibility=ReversibilityLevel.PARTIAL,
)

admin_action = ActionDescriptor(
    action_id="delete-session",
    name="Delete session",
    execute_api="/api/session/delete",
    undo_api=None,
    reversibility=ReversibilityLevel.NONE,
    is_admin=True,
)

actions = [read_action, write_action, admin_action]
print("Action → Required Ring:")
for a in actions:
    print(f"  {a.name:20s}  reversibility={a.reversibility.name:8s}  → ring {a.required_ring.name}")

In [ ]:
# Join agents with different trust scores
agents = [
    ("agent-alpha",   0.95),  # high trust
    ("agent-beta",    0.72),  # medium-high
    ("agent-gamma",   0.55),  # medium
    ("agent-delta",   0.30),  # low trust → sandbox
]

assigned_rings = {}
for did, sigma in agents:
    ring = run(hv.join_session(
        session_id=SESSION_ID,
        agent_did=did,
        actions=actions,
        sigma_raw=sigma,
    ))
    assigned_rings[did] = ring
    print(f"{did:16s}  σ={sigma:.2f}  → {ring.name}")

In [ ]:
# Ring enforcement — check if an agent can perform an action
enforcer = RingEnforcer()

print("Ring enforcement checks:")
for did, sigma in agents:
    ring = assigned_rings[did]
    for action in actions:
        result = enforcer.check(
            agent_ring=ring,
            action=action,
            eff_score=sigma,
            has_consensus=False,
            has_sre_witness=False,
        )
        status = "✅ allowed" if result.allowed else "❌ denied"
        print(f"  {did:16s} → {action.name:20s}  {status}")
    print()

---
## 3 — Saga Pattern

The saga orchestrator coordinates multi-step workflows.  
If a step fails, earlier steps are **compensated** in reverse order.

In [ ]:
# Activate the session so we can run sagas
run(hv.activate_session(SESSION_ID))
print(f"Session state: {session.session.state}")

In [ ]:
orch = session.saga

saga = orch.create_saga(session_id=SESSION_ID)
print(f"Saga created: {saga.saga_id}  state={saga.state.name}")

# Add three steps
step1 = orch.add_step(
    saga_id=saga.saga_id,
    action_id="extract",
    agent_did="agent-alpha",
    execute_api="/api/extract",
    undo_api="/api/extract/undo",
)
step2 = orch.add_step(
    saga_id=saga.saga_id,
    action_id="transform",
    agent_did="agent-beta",
    execute_api="/api/transform",
    undo_api="/api/transform/undo",
)
step3 = orch.add_step(
    saga_id=saga.saga_id,
    action_id="load",
    agent_did="agent-gamma",
    execute_api="/api/load",
    undo_api="/api/load/undo",
)

print(f"Steps: {[s.action_id for s in saga.steps]}")

In [ ]:
# Execute step 1 — succeeds
async def execute_extract():
    return {"rows": 1000, "source": "db"}

result1 = run(orch.execute_step(saga.saga_id, step1.step_id, executor=execute_extract))
print(f"Step 1 ({step1.action_id}): state={step1.state.name}  result={result1}")

In [ ]:
# Execute step 2 — succeeds
async def execute_transform():
    return {"rows_transformed": 980}

result2 = run(orch.execute_step(saga.saga_id, step2.step_id, executor=execute_transform))
print(f"Step 2 ({step2.action_id}): state={step2.state.name}  result={result2}")

In [ ]:
# Execute step 3 — FAILS, triggering compensation
async def execute_load_fail():
    raise RuntimeError("Disk full — cannot load data")

try:
    run(orch.execute_step(saga.saga_id, step3.step_id, executor=execute_load_fail))
except Exception as exc:
    print(f"Step 3 FAILED: {exc}")

print(f"\nSaga state: {saga.state.name}")
print("\nStep states after failure:")
for s in saga.steps:
    print(f"  {s.action_id:12s}  {s.state.name:20s}  error={s.error or '—'}")

---
## 4 — Kill Switch

The kill switch immediately terminates a misbehaving agent and optionally hands off
in-flight work to a substitute.

In [ ]:
ks = KillSwitch()

# Register a substitute agent
ks.register_substitute(session_id=SESSION_ID, agent_did="agent-beta")

# Kill a misbehaving agent
kill_result = ks.kill(
    agent_did="agent-delta",
    session_id=SESSION_ID,
    reason=KillReason.BEHAVIORAL_DRIFT,
    in_flight_steps=[],
    details="Trust score dropped below threshold after anomalous writes",
)

print(f"Kill ID      : {kill_result.kill_id}")
print(f"Agent        : {kill_result.agent_did}")
print(f"Reason       : {kill_result.reason.name}")
print(f"Timestamp    : {kill_result.timestamp}")
print(f"Compensation : {kill_result.compensation_triggered}")
print(f"Details      : {kill_result.details}")
print(f"\nTotal kills so far: {ks.total_kills}")

---
## 5 — Resource Limits

Each ring has a configurable rate limit. Lower-trust rings get fewer
operations per window.

In [ ]:
import time
from collections import defaultdict

# Simple per-ring rate limiter demonstration
RATE_LIMITS = {
    ExecutionRing.RING_0_ROOT:        100,  # ops / window
    ExecutionRing.RING_1_PRIVILEGED:   50,
    ExecutionRing.RING_2_STANDARD:     20,
    ExecutionRing.RING_3_SANDBOX:       5,
}

counters: dict[str, int] = defaultdict(int)
results: list[dict] = []

for did, sigma in agents:
    ring = assigned_rings[did]
    limit = RATE_LIMITS[ring]
    accepted = 0
    denied = 0
    # Simulate 25 operations
    for _ in range(25):
        counters[did] += 1
        if counters[did] <= limit:
            accepted += 1
        else:
            denied += 1
    results.append({"agent": did, "ring": ring.name, "limit": limit, "accepted": accepted, "denied": denied})

print(f"{'Agent':16s} {'Ring':22s} {'Limit':>6s} {'Accepted':>9s} {'Denied':>7s}")
print("-" * 65)
for r in results:
    print(f"{r['agent']:16s} {r['ring']:22s} {r['limit']:6d} {r['accepted']:9d} {r['denied']:7d}")

---
## 6 — Audit Trail

Every mutation is captured as a **SemanticDelta** with a hash-chained pointer,
forming a tamper-evident log.

In [ ]:
delta_engine = DeltaEngine(session_id=SESSION_ID)

# Capture a series of changes
changes_list = [
    ("agent-alpha", [
        VFSChange(path="/data/report.md", operation="create", content_hash="abc123", previous_hash=None, agent_did="agent-alpha"),
    ]),
    ("agent-beta", [
        VFSChange(path="/data/report.md", operation="update", content_hash="def456", previous_hash="abc123", agent_did="agent-beta"),
    ]),
    ("agent-gamma", [
        VFSChange(path="/data/summary.txt", operation="create", content_hash="ghi789", previous_hash=None, agent_did="agent-gamma"),
    ]),
]

deltas = []
for agent_did, changes in changes_list:
    delta = delta_engine.capture(agent_did=agent_did, changes=changes)
    deltas.append(delta)

print(f"Captured {delta_engine.turn_count} deltas\n")
for d in deltas:
    print(f"  Turn {d.turn_id}  agent={d.agent_did:16s}  hash={d.delta_hash[:16]}…  parent={str(d.parent_hash)[:16] if d.parent_hash else 'None':16s}")

# Verify chain integrity
chain_valid = delta_engine.verify_chain()
root = delta_engine.compute_hash_chain_root()
print(f"\nChain valid : {chain_valid}")
print(f"Chain root  : {root[:32] if root else 'N/A'}…")

---
## 7 — Joint Liability

Agents can **vouch** for each other by bonding part of their trust score.  
If the vouchee misbehaves, the voucher is **slashed** proportionally.

In [ ]:
vouching = VouchingEngine()
slashing = Slashing(vouching_engine=vouching)

# agent-alpha vouches for agent-gamma (bonds 20% of its sigma)
vouch = vouching.vouch(
    voucher_did="agent-alpha",
    vouchee_did="agent-gamma",
    session_id=SESSION_ID,
    voucher_sigma=0.95,
    bond_pct=0.20,
)

print(f"Vouch ID     : {vouch.vouch_id}")
print(f"Voucher      : {vouch.voucher_did}")
print(f"Vouchee      : {vouch.vouchee_did}")
print(f"Bonded σ %   : {vouch.bonded_sigma_pct:.0%}")
print(f"Bonded amount: {vouch.bonded_amount:.4f}")

# Compute effective score with the vouch backing
eff = vouching.compute_eff_score(
    vouchee_did="agent-gamma",
    session_id=SESSION_ID,
    vouchee_sigma=0.55,
    risk_weight=0.5,
)
print(f"\nEffective score for agent-gamma (with vouch): {eff:.4f}")

In [ ]:
# Now slash agent-gamma — voucher alpha gets clipped too
slash_result = slashing.slash(
    vouchee_did="agent-gamma",
    session_id=SESSION_ID,
    vouchee_sigma=0.55,
    risk_weight=0.8,
    reason="Wrote to restricted path without permission",
)

print(f"Slash ID            : {slash_result.slash_id}")
print(f"Vouchee σ before    : {slash_result.vouchee_sigma_before:.4f}")
print(f"Vouchee σ after     : {slash_result.vouchee_sigma_after:.4f}")
print(f"Reason              : {slash_result.reason}")
print(f"\nVoucher clips:")
for clip in slash_result.voucher_clips:
    print(f"  {clip}")

---
## 8 — Visualization

Charts summarising ring distribution, resource limits, and audit chain.

In [ ]:
try:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    HAS_PLOTLY = True
except ImportError:
    HAS_PLOTLY = False
    print("plotly not installed — falling back to text summaries.")
    print("Install with: pip install plotly")

In [ ]:
if HAS_PLOTLY:
    # --- Ring distribution pie chart ---
    ring_counts: dict[str, int] = defaultdict(int)
    for ring in assigned_rings.values():
        ring_counts[ring.name] += 1

    ring_colors = {
        "RING_0_ROOT": "#e74c3c",
        "RING_1_PRIVILEGED": "#f39c12",
        "RING_2_STANDARD": "#3498db",
        "RING_3_SANDBOX": "#95a5a6",
    }

    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{"type": "pie"}, {"type": "bar"}]],
        subplot_titles=("Agent Ring Distribution", "Rate Limits by Ring"),
    )

    labels = list(ring_counts.keys())
    values = list(ring_counts.values())
    colors = [ring_colors.get(l, "#bdc3c7") for l in labels]

    fig.add_trace(
        go.Pie(labels=labels, values=values, marker=dict(colors=colors), hole=0.4),
        row=1, col=1,
    )

    # --- Rate limits bar chart ---
    bar_labels = [r.name for r in RATE_LIMITS]
    bar_values = list(RATE_LIMITS.values())
    bar_colors = [ring_colors.get(l, "#bdc3c7") for l in bar_labels]

    fig.add_trace(
        go.Bar(x=bar_labels, y=bar_values, marker_color=bar_colors),
        row=1, col=2,
    )

    fig.update_layout(
        title_text="Agent Hypervisor — Overview",
        showlegend=False,
        height=400,
    )
    fig.show()
else:
    print("\n--- Ring Distribution ---")
    for did, ring in assigned_rings.items():
        print(f"  {did:16s}  {ring.name}")
    print("\n--- Rate Limits ---")
    for ring, limit in RATE_LIMITS.items():
        print(f"  {ring.name:22s}  {limit:4d} ops/window")

In [ ]:
if HAS_PLOTLY:
    # --- Trust scores vs assigned ring scatter ---
    agent_names = [a[0] for a in agents]
    sigmas = [a[1] for a in agents]
    ring_nums = [assigned_rings[a[0]].value for a in agents]

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(
        x=sigmas,
        y=ring_nums,
        mode="markers+text",
        text=agent_names,
        textposition="top center",
        marker=dict(
            size=14,
            color=sigmas,
            colorscale="RdYlGn",
            showscale=True,
            colorbar=dict(title="σ"),
        ),
    ))

    fig2.update_layout(
        title="Trust Score → Ring Assignment",
        xaxis_title="Raw Trust Score (σ)",
        yaxis_title="Execution Ring",
        yaxis=dict(
            tickvals=[0, 1, 2, 3],
            ticktext=["Ring 0 (Root)", "Ring 1 (Privileged)", "Ring 2 (Standard)", "Ring 3 (Sandbox)"],
            autorange="reversed",
        ),
        height=400,
    )
    fig2.show()
else:
    print("\n--- Trust → Ring Mapping ---")
    for did, sigma in agents:
        print(f"  σ={sigma:.2f}  →  {assigned_rings[did].name}")

In [ ]:
if HAS_PLOTLY:
    # --- Audit chain timeline ---
    fig3 = go.Figure()

    delta_agents = [d.agent_did for d in deltas]
    delta_turns = [d.turn_id for d in deltas]
    delta_hashes = [d.delta_hash[:12] + "…" for d in deltas]

    fig3.add_trace(go.Scatter(
        x=delta_turns,
        y=delta_agents,
        mode="markers+lines+text",
        text=delta_hashes,
        textposition="top center",
        marker=dict(size=12, color="#2ecc71"),
        line=dict(dash="dot", color="#27ae60"),
    ))

    fig3.update_layout(
        title="Hash-Chained Audit Trail",
        xaxis_title="Turn",
        yaxis_title="Agent",
        height=350,
    )
    fig3.show()
else:
    print("\n--- Audit Chain ---")
    for d in deltas:
        print(f"  Turn {d.turn_id}  {d.agent_did}  hash={d.delta_hash[:16]}…")

---
## Cleanup

In [ ]:
root_hash = run(hv.terminate_session(SESSION_ID))
print(f"Session terminated.")
print(f"Final hash-chain root: {root_hash}")